In [16]:
import requests
import pandas as pd

# Get the local data
# Define the API endpoint
api_url = "http://127.0.0.1:5000/api/data"

# Call the API
try:
    response = requests.get(api_url, timeout=10)

    # Check if the response is successful
    if response.status_code == 200:
        data = response.json()
        bike_df = pd.DataFrame(data)
        print("Header of the table:")
        print(bike_df.head())
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")

Header of the table:
  Bike_model  Bike_number       End_date                  End_station  \
0    CLASSIC        23715  8/1/2023 0:17  Albert Embankment, Vauxhall   
1    CLASSIC        41267  8/1/2023 0:17  Albert Embankment, Vauxhall   
2    CLASSIC        53180  8/1/2023 0:11   Baldwin Street, St. Luke's   
3    CLASSIC        53431  8/1/2023 0:12      Grosvenor Road, Pimlico   
4    CLASSIC        54758  8/1/2023 0:17      Vauxhall Walk, Vauxhall   

   End_station_number     Number     Start_date  \
0                1059  132825189  8/1/2023 0:00   
1                1059  132825190  8/1/2023 0:00   
2                3500  132825191  8/1/2023 0:00   
3                1140  132825192  8/1/2023 0:01   
4              200056  132825193  8/1/2023 0:01   

                           Start_station  Start_station_number Total_duration  \
0  Kennington Lane Rail Bridge, Vauxhall                  1190        16m 46s   
1  Kennington Lane Rail Bridge, Vauxhall                  1190        1

In [25]:
# https://api-portal.tfl.gov.uk/api-details#api=BikePoint&operation=BikePoint_GetAll

import requests
import pprint

from config import tfl_api_key

# Get the API data test
# Example API endpoint for getting bus stops near a location
# url = f"https://api.tfl.gov.uk/BikePoint/Search?query=Brunswick+Square+Bloomsbury"

# TfL API endpoint for searching bike points
url_template = "https://api.tfl.gov.uk/BikePoint/Search?query={}"
url = url_template.format('Brunswick Square, Bloomsbury')

# Set headers with your API key
headers = {"Authorization": f"AppKey {tfl_api_key}"}

response = requests.get(url, headers=headers)

# Check for successful response
if response.status_code == 200:
  data = response.json()
  # Process the data (list of bus stops)
  pprint.pprint(data)
  print(data[0]['lat'])
  print(data[0]['lon'])
else:
  print(f"Error: {response.status_code}")


[{'$type': 'Tfl.Api.Presentation.Entities.Place, Tfl.Api.Presentation.Entities',
  'additionalProperties': [],
  'children': [],
  'childrenUrls': [],
  'commonName': 'Brunswick Square, Bloomsbury',
  'id': 'BikePoints_11',
  'lat': 51.523951,
  'lon': -0.122502,
  'placeType': 'BikePoint',
  'url': '/Place/BikePoints_11'}]
51.523951
-0.122502


In [18]:
# Combine stations into a single list
all_stations = bike_df['Start_station'].tolist() + bike_df['End_station'].tolist()

# Remove duplicates and sort alphabetically
unique_stations = sorted(set(all_stations))
unique_stations

['Abbey Orchard Street, Westminster',
 "Albany Street, The Regent's Park",
 'Albert Embankment, Vauxhall',
 'Alma Road, Wandsworth',
 'Argyle Street, Kings Cross',
 "Baldwin Street, St. Luke's",
 'Bancroft Road, Bethnal Green',
 "Bath Street, St. Luke's",
 'Battersea Park Road, Nine Elms',
 'Battersea Power Station, Battersea Park',
 'Baylis Road, Waterloo',
 'Bayswater Road, Hyde Park',
 'Belgrave Road, Victoria',
 'Belvedere Road 1, South Bank',
 'Berry Street, Clerkenwell',
 'Bethnal Green Road, Shoreditch',
 'Bevington Road West, North Kensington',
 'Binfield Road, Stockwell',
 "Birkenhead Street, King's Cross",
 "Bishop's Bridge Road West, Bayswater",
 'Black Prince Road, Vauxhall',
 'Bolsover Street, Fitzrovia',
 'Bonny Street, Camden Town',
 'Braham Street, Aldgate',
 'Brandon Street, Walworth',
 'Brunswick Square, Bloomsbury',
 'Buckingham Gate, Westminster',
 'Burdett Road, Mile End',
 'Burgess Park Albany Road, Walworth',
 'Butler Place, Westminster',
 'Buxton Street 1, Shore

In [20]:

# TfL API endpoint for searching bike points
url_template = "https://api.tfl.gov.uk/BikePoint/Search?query={}"

# Function to call TfL API and parse response
def get_station_info(station_name):
  url = url_template.format(station_name)
  response = requests.get(url)
  
  if response.status_code == 200:
    data = response.json()
    # Check if there are any results
    if data['stopPoints']:
      # Assuming first stopPoint has location details
      stop_point = data['stopPoints'][0]
      location = stop_point.get('location', None)
      if location:
        # Extract latitude and longitude if available
        latitude = location.get('latitude')
        longitude = location.get('longitude')
        return pd.Series({'station_name': station_name, 'latitude': latitude, 'longitude': longitude})
      else:
        # Handle case where location data is missing
        print(f"Location data not found for {station_name}")
  else:
    print(f"Error: {response.status_code} for station {station_name}")
  return None  # Indicate no data found

# Get unique station names
stations = list(set([*bike_df['Start_station'], *bike_df['End_station']]))

# Create empty DataFrame
df_stations = pd.DataFrame(columns=['station_name', 'latitude', 'longitude'])

# Call TfL API for each station and collect data
for station in stations:
  station_info = get_station_info(station)
  if station_info is not None:
    df_stations = df_stations.append(station_info, ignore_index=True)

# Print DataFrame
print(df_stations)

TypeError: list indices must be integers or slices, not str